Consider a system of User and Feature objects, where a Feature describes the availability of
an ongoing project or change in functionality. Features may be generally available, limited to
certain regions, and/or intended for an A/B test. We need to write a deterministic system for
identifying which Features are active for a given User. We also want to write tests to ensure
that our system is working as intended each time we run it, given the example User and Feature
data below.
Users:
        [
{ "id": 0, "name": "eva",    "location": "US" },
{ "id": 1, "name": "tess",   "location": "US" },
{ "id": 2, "name": "rahool", "location": "CA" },
{ "id": 3, "name": "amanda", "location": "CA" }
    ]
Features:
        [
{
    "id": "annual_sale",
        "locations": ["US"],
    "abTest": true,
},
{
    "id": "enhanced_comments",
        "abTest": true,
},
{
    "id": "canada_promotion",
        "locations": ["CA"],
}
    ]
            ## Part 1
Write a function, get_user_features(user, features) which takes a single User object and a list
of Feature objects and returns a set of the Feature IDs that apply to the given User.
A User has three properties: an integer id, a string name, and a 2-letter country code
string location.
A Feature has three properties: a unique string id, an optional array of 2-letter country code
strings, locations, which limits the feature to users with a matching location, and an optional
boolean, abTest, which when set to true will only apply the feature to users with an even user ID.
If abTest or locations are absent for a Feature, they have no effect.
Given the features and users, the following results are expected:
        | User   | Features                            |
        | ------ | ----------------------------------- |
        | eva    | annual_sale, enhanced_comments      |
        | tess   | N/A                                 |
        | rahool | enhanced_comments, canada_promotion |
        | amanda | canada_promotion                    |
        ## Part 2
Users may want to opt-in to or opt-out of a feature. Augment get_user_features to take into
account two new properties on User objects: optIn and optOut, both of which are arrays of
feature IDs to either opt into or opt out of. A user can be opted-in to an A/B test Feature,
regardless of their ID. A user cannot be given a feature that is not available in their region,
even if they try to opt-in to it.
        Users 0 (eva),  1 (tess) and 3 (amanda) would like to opt in to the annual_sale feature.
        User 2, (rahool), would like to opt out of the enhanced_comments and canada_promotion features.
        Our new user data is...
Users:
{ "id": 0, "name": "eva",    "location": "US", "optIn": ["annual_sale"] }
{ "id": 1, "name": "tess",   "location": "US", "optIn": ["annual_sale"] }
{ "id": 2, "name": "rahool", "location": "CA", "optOut": ["enhanced_comments", "canada_promotion"] }
{ "id": 3, "name": "amanda", "location": "CA", "optIn": ["annual_sale"] }
Given these changes and updates, the following results are now expected:
        | User   | Features                       |
        | ------ | ------------------------------ |
        | eva    | annual_sale, enhanced_comments |
        | tess   | annual_sale                    |
        | rahool | N/A                            |
        | amanda | canada_promotion               |

In [1]:
def get_user_features(user, features):
    """
    Determines the active features for a given user based on feature availability, location, and A/B testing rules.

    :param user: A dictionary representing the user.
    :param features: A list of dictionaries representing the features.
    :return: A set of feature IDs that apply to the user.
    """
    active_features = set()
    user_id = user.get("id")
    user_location = user.get("location")
    opt_in = set(user.get("optIn", []))
    opt_out = set(user.get("optOut", []))

    for feature in features:
        feature_id = feature["id"]
        feature_locations = feature.get("locations", [])
        ab_test = feature.get("abTest", False)

        # Check if feature is limited to specific locations
        if feature_locations and user_location not in feature_locations:
            continue  # User's location does not qualify for this feature

        # Check A/B test eligibility (only even IDs unless opted in)
        if ab_test and user_id % 2 != 0 and feature_id not in opt_in:
            continue  # User does not qualify for A/B test feature

        # Apply feature if user is eligible and has not opted out
        if feature_id not in opt_out:
            active_features.add(feature_id)

    # Include explicitly opted-in features if they are location-compatible
    for feature_id in opt_in:
        if any(f["id"] == feature_id and (not f.get("locations") or user_location in f["locations"]) for f in features):
            active_features.add(feature_id)

    return active_features

# Example usage
users = [
    {"id": 0, "name": "eva", "location": "US", "optIn": ["annual_sale"]},
    {"id": 1, "name": "tess", "location": "US", "optIn": ["annual_sale"]},
    {"id": 2, "name": "rahool", "location": "CA", "optOut": ["enhanced_comments", "canada_promotion"]},
    {"id": 3, "name": "amanda", "location": "CA", "optIn": ["annual_sale"]},
]

features = [
    {"id": "annual_sale", "locations": ["US"], "abTest": True},
    {"id": "enhanced_comments", "abTest": True},
    {"id": "canada_promotion", "locations": ["CA"]},
]

# Compute user features
for user in users:
    print(f"{user['name']}: {get_user_features(user, features)}")

eva: {'annual_sale', 'enhanced_comments'}
tess: {'annual_sale'}
rahool: set()
amanda: {'canada_promotion'}
